In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Organización de Datos 75.06. Primer Cuatrimestre de 2017. Examen parcial, primera oportunidad

2) Un sitio de Ebooks tiene información sobre los reviews que los usuarios
hacen de sus libros en un DataFrame con formato (user_id, book_id, rating,
timestamp). Por otro lado tenemos información en otro DataFrame que bajamos
de GoodReads: (book_id, book_name, avg_rating). Podemos suponer que los
Ids de los libros son compatibles. Se pide usar Python Pandas para:

In [8]:
df_ebooks = pd.DataFrame({'user_id': pd.Series([1,2,1,1,2,4,6,4,3,3,2]),\
                   'book_id': pd.Series([1,1,2,3,2,2,1,3,1,2,3]),\
                    'rating': pd.Series([4,6,3,2,5,7,8,2,5,8,5]),\
                   'timestamp': pd.Series([20170812,20170812,20170812,20170812,20170812,20170812,20170812,20170812,20170812,20170812,20170812])})
df_ebooks

,book_id,rating,timestamp,user_id
0,1,4,20170812,1
1,1,6,20170812,2
2,2,3,20170812,1
3,3,2,20170812,1
4,2,5,20170812,2
5,2,7,20170812,4
6,1,8,20170812,6
7,3,2,20170812,4
8,1,5,20170812,3
9,2,8,20170812,3


In [5]:
df_rating = pd.DataFrame({'book_id': pd.Series([1,2,3]),\
                          'name': pd.Series(['Libro 1','Libro 2','Libro 3']),
                          'avg_rating': pd.Series([5.75,10,1])})
df_rating

,avg_rating,book_id,name
0,5.75,1,Libro 1
1,10.00,2,Libro 2
2,1.00,3,Libro 3


a) Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks.
(Para este punto se puede ignorar el segundo DataFrame). (4*) (7.5 pts)

In [60]:
grouped = df_ebooks.groupby('book_id').agg(['mean','count']).reset_index()
grouped

book_id rating       timestamp         user_id      
            mean count      mean count      mean count
0       1   5.75     4  20170812     4  3.000000     4
1       2   5.75     4  20170812     4  2.500000     4
2       3   3.00     3  20170812     3  2.333333     3

In [40]:
grouped[['book_id','rating']][grouped[('rating','count')]>1].sort_values(('rating','mean'),ascending=False)[:5]

book_id rating      
            mean count
0       1   5.75     4
1       2   5.75     4
2       3   3.00     3

a) Metodo 2

In [70]:
rating = pd.DataFrame({'rating': df_ebooks.groupby('book_id')['rating'].mean()}).reset_index()\
        .sort_values('rating',ascending=False)
rating[:5]

,book_id,rating
0,1,5.75
1,2,5.75
2,3,3.00


b) Obtener un DataFrame que indique qué libros tienen una diferencia de rating
promedio mayor al 20% entre el sitio de Ebooks y GoodReads. (3*) (7.5 pts)

In [90]:
mg = pd.merge(rating,df_rating, left_on='book_id',right_on='book_id',how='inner')
mg

,book_id,rating,avg_rating,name
0,1,5.75,5.75,Libro 1
1,2,5.75,10.00,Libro 2
2,3,3.00,1.00,Libro 3


In [96]:
# calculo el porcentaje con respecto a GoodReads
mg['diff'] = abs(1- mg['rating'] / mg['avg_rating'])
# me fijo cuales tienen un porcentaje mayor a 20%
mg[mg['diff']>0.2]

,book_id,rating,avg_rating,name,diff
1,2,5.75,10.0,Libro 2,0.425
2,3,3.00,1.0,Libro 3,2.000


# Organización de Datos 75.06. Primer Cuatrimestre de 2017. Examen parcial, segunda oportunidad

a) Datafram con (from_airport, to_airport, airline, delay). Cuales son las 10 rutas con mayor promedio de demoras. Indicar origen, destino y demora promedio. Ejemplo (ATL-DEL, 15)

In [17]:
df_flights = pd.DataFrame({'from_airport': pd.Series(['ATL','ATL','DAL','ATL','DAL','NYC','NYC','NYC','ATL','ATL']),\
                            'to_airport': pd.Series(['NYC','DAL','NYC','DAL','ATL','ATL','ATL','ATL','NYC','DAL']),\
                            'airline': pd.Series(['a','b','a','b','b','c','a','b','b','c']),\
                            'delay': pd.Series([0,2,5,3,2,0,2,5,3,2])})
df_flights

,airline,delay,from_airport,to_airport
0,a,0,ATL,NYC
1,b,2,ATL,DAL
2,a,5,DAL,NYC
3,b,3,ATL,DAL
4,b,2,DAL,ATL
5,c,0,NYC,ATL
6,a,2,NYC,ATL
7,b,5,NYC,ATL
8,b,3,ATL,NYC
9,c,2,ATL,DAL


In [18]:
#una ruta esta compuesta por from - to
df_flights['route'] = df_flights['from_airport']+'-'+df_flights['to_airport']
flights = df_flights[['route','delay','airline']]
flights

,route,delay,airline
0,ATL-NYC,0,a
1,ATL-DAL,2,b
2,DAL-NYC,5,a
3,ATL-DAL,3,b
4,DAL-ATL,2,b
5,NYC-ATL,0,c
6,NYC-ATL,2,a
7,NYC-ATL,5,b
8,ATL-NYC,3,b
9,ATL-DAL,2,c


In [23]:
stats_flights = flights[['route','delay']].groupby('route').agg(['mean','size'])
stats_flights

delay     
             mean size
route                 
ATL-DAL  2.333333    3
ATL-NYC  1.500000    2
DAL-ATL  2.000000    1
DAL-NYC  5.000000    1
NYC-ATL  2.333333    3

In [69]:
avg_delay_flights = stats_flights[stats_flights[('delay','size')]>1].sort_values(('delay','mean'),ascending=False).reset_index()
avg_by_route = avg_delay_flights.assign(avg=avg_delay_flights[('delay','mean')])[['route','avg']][:10]
avg_by_route = pd.DataFrame({'route': avg_by_route['route'], 'avg': avg_by_route['avg']})
avg_by_route

,avg,route
0,2.333333,ATL-DAL
1,2.333333,NYC-ATL
2,1.500000,ATL-NYC


b) Para las 10 rutas anteriores, cual es la linea que tiene mayor promedio de demoras para las mismas

In [73]:
mg = pd.merge(avg_by_route,flights,left_on='route',right_on='route',how='left')
mg

,avg,route,delay,airline
0,2.333333,ATL-DAL,2,b
1,2.333333,ATL-DAL,3,b
2,2.333333,ATL-DAL,2,c
3,2.333333,NYC-ATL,0,c
4,2.333333,NYC-ATL,2,a
5,2.333333,NYC-ATL,5,b
6,1.500000,ATL-NYC,0,a
7,1.500000,ATL-NYC,3,b


In [72]:
mg[0:1]['airline']

0    b
Name: airline, dtype: object